In [12]:

import pandas as pd
import os
import json

from geopy.geocoders import Nominatim
import folium

In [13]:
# Using file from Main Reference website
parking_df = pd.read_csv('http://iot.ee.surrey.ac.uk:8080/datasets/parking/aarhus_parking.csv', sep=',')
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44
2,869,2014-05-22 09:09:04.145,3,1240,SCANDCENTER,2014-11-03 16:18:44
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44


In [14]:
# update garage code to geolocation search to work
parking_df.loc[parking_df['garagecode'] == 'SCANDCENTER', 'garagecode'] = 'SCANDINAVIAN CENTER'
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44
2,869,2014-05-22 09:09:04.145,3,1240,SCANDINAVIAN CENTER,2014-11-03 16:18:44
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44


# Process geolocation from garageCodes using GeoPy


In [15]:

garages = parking_df.groupby("garagecode")
print(garages.count())

garage_codes = garages['garagecode'].unique()
print(garage_codes)


# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
for row in garage_codes:
    getLoc = loc.geocode(f'{row[0]}, Aarhus, Denmark')
    parking_df.loc[parking_df['garagecode'] == row[0], 'latitude'] = getLoc.latitude
    parking_df.loc[parking_df['garagecode'] == row[0],'longitude'] = getLoc.longitude
    parking_df.loc[parking_df['garagecode'] == row[0],'address'] = getLoc.address
    

                     vehiclecount  updatetime   _id  totalspaces  streamtime
garagecode                                                                  
BRUUNS                       6908        6908  6908         6908        6908
BUSGADEHUSET                 6908        6908  6908         6908        6908
KALKVAERKSVEJ                6908        6908  6908         6908        6908
MAGASIN                      6908        6908  6908         6908        6908
NORREPORT                    6908        6908  6908         6908        6908
SALLING                      6908        6908  6908         6908        6908
SCANDINAVIAN CENTER          6908        6908  6908         6908        6908
SKOLEBAKKEN                  6908        6908  6908         6908        6908
garagecode
BRUUNS                              [BRUUNS]
BUSGADEHUSET                  [BUSGADEHUSET]
KALKVAERKSVEJ                [KALKVAERKSVEJ]
MAGASIN                            [MAGASIN]
NORREPORT                        [NORRE

In [16]:
parking_df.head()

,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime,latitude,longitude,address
0,0,2014-05-22 09:09:04.145,1,65,NORREPORT,2014-11-03 16:18:44,56.160634,10.212609,"Nørreport, Aarhus Ø, Aarhus, Aarhus Kommune, R..."
1,0,2014-05-22 09:09:04.145,2,512,SKOLEBAKKEN,2014-11-03 16:18:44,56.157146,10.213121,"Skolebakken, Aarhus, Aarhus Kommune, Region Mi..."
2,869,2014-05-22 09:09:04.145,3,1240,SCANDINAVIAN CENTER,2014-11-03 16:18:44,56.152770,10.197070,"Scandinavian Center Aarhus, Skovgaardsgade, Fr..."
3,22,2014-05-22 09:09:04.145,4,953,BRUUNS,2014-11-03 16:18:44,56.149019,10.204733,"Bruuns Galleri, Værkmestergade, Midtbyen, Fred..."
4,124,2014-05-22 09:09:04.145,5,130,BUSGADEHUSET,2014-11-03 16:18:44,56.155552,10.206232,"Busgadehuset, Telefontorvet, Midtbyen, Frederi..."


In [22]:
na_rows = parking_df[parking_df.isna().any(axis=1)]
na_rows

,Unnamed: 0,vehiclecount,updatetime,_id,totalspaces,garagecode,streamtime,latitude,longitude,address


In [ ]:
# This is the file we will use in our analyze 
parking_df.to_csv('../data/aarhus_parking_geolocated.csv')

## Georeferencing


In [18]:
AARHUS_GEOLOCATION = [56.16182966470494, 10.197421860666816]

parking_df = pd.read_csv('../data/aarhus_parking_geolocated.csv', sep=',')
map = folium.Map(location=AARHUS_GEOLOCATION, zoom_start=13)
garage_codes = parking_df.groupby(['garagecode', 'address', 'latitude', 'longitude']).count().reset_index()


garage_codes.head()

,garagecode,address,latitude,longitude,Unnamed: 0,vehiclecount,updatetime,_id,totalspaces,streamtime
0,BRUUNS,"Bruuns Galleri, Værkmestergade, Midtbyen, Fred...",56.149019,10.204733,6908,6908,6908,6908,6908,6908
1,BUSGADEHUSET,"Busgadehuset, Telefontorvet, Midtbyen, Frederi...",56.155552,10.206232,6908,6908,6908,6908,6908,6908
2,KALKVAERKSVEJ,"Kalkværksvej, Sydhavnskvarteret, Frederiksbjer...",56.148237,10.211333,6908,6908,6908,6908,6908,6908
3,MAGASIN,"Magasin, Åboulevarden, Midtbyen, Aarhus, Aarhu...",56.156939,10.206588,6908,6908,6908,6908,6908,6908
4,NORREPORT,"Nørreport, Aarhus Ø, Aarhus, Aarhus Kommune, R...",56.160634,10.212609,6908,6908,6908,6908,6908,6908


In [19]:
# markers on a map
for index, address in garage_codes.iterrows():
   folium.Marker(
         location=[address['latitude'], address['longitude']],
         popup=address['garagecode'],
      ).add_to(map)

map